# Manually check the counties in the BAs that are shared by multiple loadzones.

In [3]:
import pandas as pd
import numpy as np
from functools import reduce
from collections import defaultdict

In [4]:
#Loadzone to BA map, based on Balancing_Authorities_JAN17.pdf
loadzone_ba = {
    'WA' : ['AVA','BPAT','CHPD','DOPD','GCPD','PSEI','SCL','TPWR'],
    'OR' : ['BPAT','GRID','IPCO','PACW','PGE'],
    'CAnorth' : ['BANC','CISO','PACW','BPAT'],
    'CABayArea' : ['CISO'],
    'CAcentral' : ['CISO', 'TIDC'],
    'CAsw' : ['CISO', 'LDWP'],
    'CAse' : ['CISO', 'IID'],
    'NV' : ['NEVP'],
    'AZ' : ['AZPS','DEAA','GRIF','GRMA','HGMA','PNM','SRP','TEPC','WALC'],
    'UT' : ['PACE'],
    'NM' : ['EPE','PNM'],
    'CO' : ['PSCO','WACM'],
    'WY' : ['PACE','WACM'],
    'ID' : ['AVA','BPAT','IPCO','PACE'],
    'MT' : ['BPAT','GWA','NWMT','WAUW','WWA'],
    'ElPaso' : ['EPE']
}

# Decompose demand of each BA that serves more than one loadzone proportional to the total population of covered counties in each corresponding loadzone

In [5]:
#Get BA loadzone map
ba_loadzone = defaultdict(set)
for loadzone,ba_list in loadzone_ba.items():
    for ba in ba_list:
        ba_loadzone[ba].add(loadzone)
ba_loadzone

defaultdict(set,
            {'AVA': {'ID', 'WA'},
             'BPAT': {'CAnorth', 'ID', 'MT', 'OR', 'WA'},
             'CHPD': {'WA'},
             'DOPD': {'WA'},
             'GCPD': {'WA'},
             'PSEI': {'WA'},
             'SCL': {'WA'},
             'TPWR': {'WA'},
             'GRID': {'OR'},
             'IPCO': {'ID', 'OR'},
             'PACW': {'CAnorth', 'OR'},
             'PGE': {'OR'},
             'BANC': {'CAnorth'},
             'CISO': {'CABayArea', 'CAcentral', 'CAnorth', 'CAse', 'CAsw'},
             'TIDC': {'CAcentral'},
             'LDWP': {'CAsw'},
             'IID': {'CAse'},
             'NEVP': {'NV'},
             'AZPS': {'AZ'},
             'DEAA': {'AZ'},
             'GRIF': {'AZ'},
             'GRMA': {'AZ'},
             'HGMA': {'AZ'},
             'PNM': {'AZ', 'NM'},
             'SRP': {'AZ'},
             'TEPC': {'AZ'},
             'WALC': {'AZ'},
             'PACE': {'ID', 'UT', 'WY'},
             'EPE': {'ElPaso', 'NM'},
      

### Get inaccurate BA county table through the two utilities forms and plot them on the map, then do a manually correction

In [6]:
#Get inaccurate BA county table
dir_files = '../test/data/'

county_utility_file = 'Service_Territory_2016.xlsx'  #get utility-county mapping
ba_utility_file = 'Sales_Ult_Cust_2016.xlsx'  #Get utility-BA mapping
# ba_list_file = 'Balancing_Authority_2016.xlsx'  #list of BAs by state

county_utility =  pd.read_excel(io = dir_files + '/' + county_utility_file, 
                                   dtype = {'Utility Number': str, 'County': str, 'State': str}
                               )
ba_utility = pd.read_excel(io = dir_files + '/' + ba_utility_file, 
                          usecols = 'B,G,I,W', skiprows = [0,1], 
                          dtype = {'Utility Number': str, 'BA_CODE': str, 'State': str},
                          sheet_name = 'States'
                          )
ba_county = pd.merge(county_utility, ba_utility, how='outer', on ='Utility Number')

#Cleaning
ba_county_uniq = ba_county[['BA_CODE','County','State_x','State_y']].drop_duplicates(keep='first')
ba_county_uniq.drop(columns = ['State_y'], inplace=True)
ba_county_uniq.rename(columns = {'State_x':'State'}, inplace=True)
ba_county_uniq.dropna(axis = 0, how = 'any', inplace=True)
ba_county_uniq.drop_duplicates(keep = 'first', inplace=True)

In [7]:
AVA_county = ba_county_uniq[ba_county_uniq['BA_CODE']=='AVA']
BPAT_county = ba_county_uniq[ba_county_uniq['BA_CODE']=='BPAT']
IPCO_county = ba_county_uniq[ba_county_uniq['BA_CODE']=='IPCO']
PACW_county = ba_county_uniq[ba_county_uniq['BA_CODE']=='PACW']
PNM_county = ba_county_uniq[ba_county_uniq['BA_CODE']=='PNM']
PACE_county = ba_county_uniq[ba_county_uniq['BA_CODE']=='PACE']
EPE_county = ba_county_uniq[ba_county_uniq['BA_CODE']=='EPE']
WACM_county = ba_county_uniq[ba_county_uniq['BA_CODE']=='WACM']

# NEVP_county = ba_county_uniq[ba_county_uniq['BA_CODE']=='NEVP']
# WALC_county = ba_county_uniq[ba_county_uniq['BA_CODE']=='WALC']

In [8]:
#Plot on the counties of a BA on the map
from bokeh.io import show,output_notebook,output_file,export
from bokeh.models.widgets import Panel, Tabs
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper,
    CustomJS,
    Slider,
    CheckboxGroup,
    Select,
    Div,
    TapTool,
    ColorBar,
    LinearColorMapper
)
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure
from bokeh.layouts import (widgetbox, row, gridplot)

from bokeh.tile_providers import CARTODBPOSITRON,STAMEN_TONER_BACKGROUND,STAMEN_TONER,STAMEN_TERRAIN

# import bokeh.sampledata
# bokeh.sampledata.download()
from bokeh.sampledata.us_counties import data as us_counties
from bokeh.sampledata.us_states import data as us_states
from bokeh import events

import matplotlib.pyplot as plt
from bokeh.colors.groups import yellow
import bokeh.colors

import seaborn as sns
import matplotlib.cm as cm
import matplotlib

In [9]:
state_xs = [us_states[state]["lons"] for state in us_states if state not in {'HI','AK'}]
state_ys = [us_states[state]["lats"] for state in us_states if state not in {'HI','AK'}]

In [10]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}
# us_state_abbrev = set(us_state_abbrev.values())

In [11]:
target_states = {'WA','ID'}
AVA_county_filtered_by_loadzone = set()
for index, row in AVA_county.iterrows():
    if row['State'] in target_states:
        AVA_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())
        
target_states = {'WA','ID','MT','OR','CA'}
# target_states = us_state_abbrev
BPAT_county_filtered_by_loadzone = set()
for index, row in BPAT_county.iterrows():
    if row['State'] in target_states:
        BPAT_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())
        
target_states = {'ID','OR'}
IPCO_county_filtered_by_loadzone = set()
for index, row in IPCO_county.iterrows():
    if row['State'] in target_states:
        IPCO_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())
        
target_states = {'CA','OR'}
# target_states = us_state_abbrev
PACW_county_filtered_by_loadzone = set()
for index, row in PACW_county.iterrows():
    if row['State'] in target_states:
        PACW_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())
        
target_states = {'AZ','NM'}
PNM_county_filtered_by_loadzone = set()
for index, row in PNM_county.iterrows():
    if row['State'] in target_states:
        PNM_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())
        
target_states = {'ID','UT','WY'}
# target_states = us_state_abbrev
PACE_county_filtered_by_loadzone = set()
for index, row in PACE_county.iterrows():
    if row['State'] in target_states:
        PACE_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())

target_states = {'NM','TX'}
EPE_county_filtered_by_loadzone = set()
for index, row in EPE_county.iterrows():
    if row['State'] in target_states:
        EPE_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())
        
target_states = {'CO','WY'}
WACM_county_filtered_by_loadzone = set()
for index, row in WACM_county.iterrows():
    if row['State'] in target_states:
        WACM_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())

# target_states = us_state_abbrev
# NEVP_county_filtered_by_loadzone = set()
# for index, row in NEVP_county.iterrows():
#     if row['State'] in target_states:
#         NEVP_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())
        
# target_states = us_state_abbrev
# WALC_county_filtered_by_loadzone = set()
# for index, row in WALC_county.iterrows():
#     if row['State'] in target_states:
#         WALC_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())

In [107]:
ba_loadzone_county_split = defaultdict(list)

#AVA################################################################
ba_loadzone_county_split['AVA_WA'] = ['Ferry_wa', 'Stevens_wa', 'Pend Oreille_wa','Lincoln_wa','Spokane_wa','Whitman_wa']
ba_loadzone_county_split['AVA_ID'] = ['Bonner_id','Kootenai_id','Benewah_id','Latah_id']    
#BPAT###############################################################
#Note: the region of SCL is less than a county, IGNORED here, keep King County in BPAT_WA
GCPD = ['Grant_wa']
DOPD = ['Okanogan_wa','Douglas_wa']
CHPD = ['Chelan_wa']
PSEI = ['Whatcom_wa','Skagit_wa']
TPWR = ['Thurston_wa']
AVA_WA = ['Ferry_wa', 'Stevens_wa', 'Pend Oreille_wa','Lincoln_wa','Spokane_wa','Whitman_wa']
for county in BPAT_county_filtered_by_loadzone:
    if county[-2:] == 'wa' and county not in set(GCPD+DOPD+CHPD+PSEI+TPWR+AVA_WA):
        ba_loadzone_county_split['BPAT_WA'].append(county)
ba_loadzone_county_split['BPAT_WA'].append('King_wa')

ba_loadzone_county_split['BPAT_CAnorth'] = ['Modoc_ca']
ba_loadzone_county_split['BPAT_ID'] = ['Shoshone_id','Clearwater_id','Lewis_id','Nez Perce_id','Boundary_id']
ba_loadzone_county_split['BPAT_MT'] = ['Lincoln_mt','Flathead_mt','Lake_mt','Sanders_mt','Mineral_mt']

PACW_OR = ['Douglas_or','Coos_or','Josephine_or','Jackson_or','Klamath_or','Curry_or']
PGE = ['Washington_or','Yamhill_or','Polk_or','Marion_or','Clackamas_or','Multnomah_or','Hood River_or']
GRID = ['Morrow_or']
for county in BPAT_county_filtered_by_loadzone:
    if county[-2:] == 'or' and county not in set(PGE+GRID+PACW_OR):
        ba_loadzone_county_split['BPAT_OR'].append(county)

#IPCO###############################################################
ba_loadzone_county_split['IPCO_OR'] = ['Malheur_or']
for county in IPCO_county_filtered_by_loadzone:
    if county[-2:] == 'id':
        ba_loadzone_county_split['IPCO_ID'].append(county)
ba_loadzone_county_split['IPCO_ID'].append('Butte_id')
ba_loadzone_county_split['IPCO_ID'].remove('Oneida_id')
ba_loadzone_county_split['IPCO_ID'].remove('Bannock_id')
ba_loadzone_county_split['IPCO_ID'].remove('Power_id')
ba_loadzone_county_split['IPCO_ID'].remove('Bingham_id')

#PACW################################################################
ba_loadzone_county_split['PACW_OR'] = ['Douglas_or','Coos_or','Josephine_or','Jackson_or','Klamath_or','Curry_or']
ba_loadzone_county_split['PACW_CAnorth'] = ['Del Norte_ca','Siskiyou_ca']

#PNM#################################################################
ba_loadzone_county_split['PNM_AZ'].append('Greenlee_az')
for county in PNM_county_filtered_by_loadzone:
    if county[-2:] == 'nm':
        ba_loadzone_county_split['PNM_NM'].append(county)
ba_loadzone_county_split['PNM_NM'].append('Lea_nm')
ba_loadzone_county_split['PNM_NM'].append('Roosevelt_nm')
ba_loadzone_county_split['PNM_NM'].append('Curry_nm')
ba_loadzone_county_split['PNM_NM'].remove('Dona Ana_nm')
ba_loadzone_county_split['PNM_NM'].remove('Otero_nm')

#PACE################################################################
for county in PACE_county_filtered_by_loadzone:
    if county[-2:] == 'ut':
        ba_loadzone_county_split['PACE_UT'].append(county)
ba_loadzone_county_split['PACE_UT'].append('Kane_ut')
ba_loadzone_county_split['PACE_UT'].append('Wayne_ut')
ba_loadzone_county_split['PACE_UT'].append('Daggett_ut')

for county in PACE_county_filtered_by_loadzone:
    if county[-2:] == 'wy':
        ba_loadzone_county_split['PACE_WY'].append(county)
ba_loadzone_county_split['PACE_WY'].remove('Albany_wy')
ba_loadzone_county_split['PACE_WY'].remove('Platte_wy')
ba_loadzone_county_split['PACE_WY'].remove('Converse_wy')
ba_loadzone_county_split['PACE_WY'].remove('Natrona_wy')
ba_loadzone_county_split['PACE_WY'].remove('Johnson_wy')
ba_loadzone_county_split['PACE_WY'].append('Teton_wy')

for county in PACE_county_filtered_by_loadzone:
    if county[-2:] == 'id':
        ba_loadzone_county_split['PACE_ID'].append(county)
ba_loadzone_county_split['PACE_ID'].remove('Butte_id')

#WACM################################################################
for county in WACM_county_filtered_by_loadzone:
    if county[-2:] == 'wy':
        if county not in ba_loadzone_county_split['PACE_WY']:
            ba_loadzone_county_split['WACM_WY'].append(county)
# del ba_loadzone_county_split['PACE_WY']

PSCO = ['Alamosa_co','Costilla_co','Huerfano_co','Las Animas_co','Baca_co','Prowers_co','Bent_co','Otero_co','Pueblo_co','Crowley_co','Lincoln_co','El Paso_co','Teller_co','Douglas_co','Elbert_co','Jefferson_co','Denver_co','Arapahoe_co','Adams_co','Clear Creek_co','Gilpin_co','Boulder_co','Weld_co','Morgan_co','Broomfield_co']
for county in WACM_county_filtered_by_loadzone:
    if county[-2:] == 'co':
        if county not in PSCO:
            ba_loadzone_county_split['WACM_CO'].append(county)
ba_loadzone_county_split['WACM_CO'].append('Eagle_co')
ba_loadzone_county_split['WACM_CO'].append('Park_co')

#EPE#################################################################
ba_loadzone_county_split['EPE_NM'] = ['Dona Ana_nm','Otero_nm','Eddy_nm']
ba_loadzone_county_split['EPE_ElPaso'] = ['El Paso_tx','Hudspeth_tx','Culberson_tx']

#CISO################################################################
# 'CISO': {'CABayArea', 'CAcentral', 'CAnorth', 'CAse', 'CAsw'}
    
ba_loadzone_county_split['CISO_CAnorth'] = ['Butte','Colusa','Del Norte','El Dorado','Glenn','Humboldt',
                       'Lake','Lassen','Mendocino','Modoc','Nevada','Placer','Plumas',
                       'Sacramento','Shasta','Sierra','Siskiyou','Sutter','Tehama',
                       'Trinity','Yolo','Yuba']
ba_loadzone_county_split['CISO_CABayArea'] = ['Alameda','Contra Costa','Marin','Napa','San Francisco','San Mateo',
            'Santa Clara','Santa Cruz','Solano','Sonoma']
ba_loadzone_county_split['CISO_CAcentral'] = ['Alpine','Amador','Calaveras','Fresno','Inyo','Kings','Madera',
                      'Mariposa','Merced','Mono','Monterey','San Benito','San Joaquin',
                      'Stanislaus','Tulare','Tuolumne']
ba_loadzone_county_split['CISO_CAse'] = ['Imperial','Orange','Riverside','San Bernardino','San Diego']
ba_loadzone_county_split['CISO_CAsw'] = ['Kern','Los Angeles','San Luis Obispo','Santa Barbara','Ventura']

for ba_loadzone in ba_loadzone_county_split:
    if ba_loadzone.split('_')[0] == 'CISO':
        for i,county in enumerate(ba_loadzone_county_split[ba_loadzone]):
            ba_loadzone_county_split[ba_loadzone][i] = county+'_'+'ca'
# ba_loadzone_county_split

In [108]:
counties_xs = []
counties_ys = []
counties_name = []
for code in us_counties:
    key = str(us_counties[code]['name'])+'_'+str(us_counties[code]['state'])
#     if key in AVA_county_filtered_by_loadzone:
#     if key in BPAT_county_filtered_by_loadzone:
#     if key in IPCO_county_filtered_by_loadzone:
#     if key in PACW_county_filtered_by_loadzone:
#     if key in PNM_county_filtered_by_loadzone:
#     if key in PACE_county_filtered_by_loadzone:
#     if key in EPE_county_filtered_by_loadzone:
#     if key in WACM_county_filtered_by_loadzone:

#     if key in PSCO_county_filtered_by_loadzone:
#     if key in NEVP_county_filtered_by_loadzone:
#     if key in WALC_county_filtered_by_loadzone:
    if key in reduce(lambda x,y:x+y,list(ba_loadzone_county_split.values())):
        counties_xs.append(us_counties[code]['lons'])
        counties_ys.append(us_counties[code]['lats'])
        counties_name.append(key)
source = ColumnDataSource(data=dict(
    x = counties_xs,
    y = counties_ys,
    name = counties_name
#     color = 'gray'
))

In [109]:
p = figure(title="BA County Check", 
           toolbar_location="left", plot_width=1300, plot_height=700,
           tools = ['tap','box_zoom','wheel_zoom','save','reset'])

p.patches(state_xs, state_ys, fill_alpha=0,
    line_color="gray", line_alpha=0.5, line_width=1.0)
# p.patches(counties_xs, counties_ys, fill_alpha=0.5,
#     line_color="gray", line_alpha=0.5, line_width=0.5)

county_render = p.patches('x','y', source = source, fill_alpha=0.5,
    line_color="gray", line_alpha=0.5, line_width=0.5)

hover1 = HoverTool(renderers=[county_render],name='Test')
hover1.point_policy = "follow_mouse"
hover1.tooltips = [
    ("County", "@name")
]
p.add_tools(hover1) 

In [15]:
show(p)

In [378]:
ba_loadzone_county_split            

defaultdict(list,
            {'AVA_WA': ['Ferry_wa',
              'Stevens_wa',
              'Pend Oreille_wa',
              'Lincoln_wa',
              'Spokane_wa',
              'Whitman_wa'],
             'AVA_ID': ['Bonner_id', 'Kootenai_id', 'Benewah_id', 'Latah_id'],
             'BPAT_WA': ['Cowlitz_wa',
              'Snohomish_wa',
              'Skamania_wa',
              'Mason_wa',
              'San Juan_wa',
              'Island_wa',
              'Clark_wa',
              'Pierce_wa',
              'Jefferson_wa',
              'Klickitat_wa',
              'Kittitas_wa',
              'Adams_wa',
              'Kitsap_wa',
              'Lewis_wa',
              'Franklin_wa',
              'Grays Harbor_wa',
              'Columbia_wa',
              'Garfield_wa',
              'Clallam_wa',
              'Asotin_wa',
              'Walla Walla_wa',
              'Benton_wa',
              'Yakima_wa',
              'Pacific_wa',
              'Wahkiakum_wa',
 

### Get fraction parameters for BAs in each loadzone via county population

In [110]:
us_counties_pop_file = 'USCountyPop.csv'
counties_pop = pd.read_csv(dir_files + '/' + us_counties_pop_file, header=0)
#cleaning
counties_pop.drop(columns = ['GEO.id', 'GEO.id2', 'rescen42010', 'resbase42010',
       'respop72010', 'respop72011', 'respop72012', 'respop72013',
       'respop72014', 'respop72015','respop72017'], inplace = True)
counties_pop['County'] = counties_pop['GEO.display-label'].str.split(',',expand=True)[0].str.replace(' County','')
counties_pop['State'] = counties_pop['GEO.display-label'].str.split(',',expand=True)[1].str.replace('^ ','')
counties_pop['State'] = counties_pop['State'].apply(lambda x: us_state_abbrev[x])
counties_pop.drop(columns = 'GEO.display-label', inplace = True)
counties_pop.rename(columns = {'respop72016':'pop2016'}, inplace=True)

In [111]:
ba_loadzone_frac = defaultdict(int)
for ba,counties in ba_loadzone_county_split.items():
    for county in counties:
        c = county.split('_')
#         print(c)
        if c[0] == 'Dona Ana':
            c[0] = 'Doña Ana'
        ba_loadzone_frac[ba] += int(counties_pop.loc[(counties_pop['County']==c[0]) & (counties_pop['State']==c[1].upper())].iloc[0]['pop2016'])
ba_total = defaultdict(int)
for ba,pop in ba_loadzone_frac.items():
    ba_total[ba.split('_')[0]] += pop
for ba in ba_loadzone_frac:
    name = ba.split('_')[0]
    ba_loadzone_frac[ba] /= ba_total[name]
ba_loadzone_frac

defaultdict(int,
            {'AVA_WA': 0.7185583026116248,
             'AVA_ID': 0.2814416973883752,
             'BPAT_WA': 0.8023965763195435,
             'BPAT_CAnorth': 0.0012383141974710193,
             'BPAT_ID': 0.010603022035095494,
             'BPAT_MT': 0.0224679029957204,
             'BPAT_OR': 0.1632941844521696,
             'IPCO_OR': 0.029817258554569813,
             'IPCO_ID': 0.9701827414454302,
             'PACW_OR': 0.8876045179305965,
             'PACW_CAnorth': 0.11239548206940354,
             'PNM_AZ': 0.005472149924283824,
             'PNM_NM': 0.9945278500757162,
             'PACE_UT': 0.8356703007903719,
             'PACE_WY': 0.06242051469556837,
             'PACE_ID': 0.10190918451405982,
             'WACM_WY': 0.24776448949692959,
             'WACM_CO': 0.7522355105030705,
             'EPE_NM': 0.28519271530306717,
             'EPE_ElPaso': 0.7148072846969329,
             'CISO_CAnorth': 0.08950746626745869,
             'CISO_CABayArea': 

In [112]:
loadzone_ba_rename_by_frac = defaultdict(list)
for lz,bas in loadzone_ba.items():
    for ba in bas:
        name = ba + '_' + lz
        if name in ba_loadzone_frac:
            loadzone_ba_rename_by_frac[lz].append(name)
        else:
            loadzone_ba_rename_by_frac[lz].append(ba)
loadzone_ba_rename_by_frac

defaultdict(list,
            {'WA': ['AVA_WA',
              'BPAT_WA',
              'CHPD',
              'DOPD',
              'GCPD',
              'PSEI',
              'SCL',
              'TPWR'],
             'OR': ['BPAT_OR', 'GRID', 'IPCO_OR', 'PACW_OR', 'PGE'],
             'CAnorth': ['BANC',
              'CISO_CAnorth',
              'PACW_CAnorth',
              'BPAT_CAnorth'],
             'CABayArea': ['CISO_CABayArea'],
             'CAcentral': ['CISO_CAcentral', 'TIDC'],
             'CAsw': ['CISO_CAsw', 'LDWP'],
             'CAse': ['CISO_CAse', 'IID'],
             'NV': ['NEVP'],
             'AZ': ['AZPS',
              'DEAA',
              'GRIF',
              'GRMA',
              'HGMA',
              'PNM_AZ',
              'SRP',
              'TEPC',
              'WALC'],
             'UT': ['PACE_UT'],
             'NM': ['EPE_NM', 'PNM_NM'],
             'CO': ['PSCO', 'WACM_CO'],
             'WY': ['PACE_WY', 'WACM_WY'],
             'ID': ['AVA

### Get demand profile in loadzones from BA profiles based on the calculated fractions

In [113]:
ba_demand_file = 'BA_fixed_demand_threshold_3.csv'
ba_demand = pd.read_csv(dir_files + '/' + ba_demand_file, header = 0, parse_dates = True, index_col = 'UTC Time')
ba_demand.fillna(0, inplace=True)

In [114]:
from powersimdata.input.grid import Grid

In [115]:
western = Grid(['Western'])

--> Loading Western interconnect
Loading zone
Loading sub
Loading bus2sub
Loading bus
Loading plant
Loading plant cost
Loading branch
Loading DC line


In [116]:
# Index verification
western.id2zone

{201: 'Washington',
 202: 'Oregon',
 203: 'Northern California',
 204: 'Bay Area',
 205: 'Central California',
 206: 'Southwest California',
 207: 'Southeast California',
 208: 'Nevada',
 209: 'Arizona',
 210: 'Utah',
 211: 'New Mexico Western',
 212: 'Colorado',
 213: 'Wyoming',
 214: 'Idaho',
 215: 'Montana Western',
 216: 'El Paso'}

In [117]:
loadzone_ba_rename_by_frac

defaultdict(list,
            {'WA': ['AVA_WA',
              'BPAT_WA',
              'CHPD',
              'DOPD',
              'GCPD',
              'PSEI',
              'SCL',
              'TPWR'],
             'OR': ['BPAT_OR', 'GRID', 'IPCO_OR', 'PACW_OR', 'PGE'],
             'CAnorth': ['BANC',
              'CISO_CAnorth',
              'PACW_CAnorth',
              'BPAT_CAnorth'],
             'CABayArea': ['CISO_CABayArea'],
             'CAcentral': ['CISO_CAcentral', 'TIDC'],
             'CAsw': ['CISO_CAsw', 'LDWP'],
             'CAse': ['CISO_CAse', 'IID'],
             'NV': ['NEVP'],
             'AZ': ['AZPS',
              'DEAA',
              'GRIF',
              'GRMA',
              'HGMA',
              'PNM_AZ',
              'SRP',
              'TEPC',
              'WALC'],
             'UT': ['PACE_UT'],
             'NM': ['EPE_NM', 'PNM_NM'],
             'CO': ['PSCO', 'WACM_CO'],
             'WY': ['PACE_WY', 'WACM_WY'],
             'ID': ['AVA

In [118]:
western_demand_v4 = pd.DataFrame(index = ba_demand.index.copy())

In [119]:
for i in range(201,217):
    western_demand_v4[i] = 0
col = 200
for lz,bas in loadzone_ba_rename_by_frac.items():
    col += 1
    for ba in bas:
        if ba in ba_loadzone_frac:
            ba_name = ba.split('_')[0]
            fraction = ba_loadzone_frac[ba]
            western_demand_v4[col] += ba_demand[ba_name]*fraction
        else:
            western_demand_v4[col] += ba_demand[ba]

In [120]:
western_demand_v4.head()

,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216
UTC Time,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,14957.128079,6735.530074,4364.428099,4828.974531,2645.481897,10642.839410,7083.349376,3910.0,8183.390209,4765.827725,1948.453213,8256.774351,1162.209844,3167.700908,1766.755706,606.156577
2016-01-01 01:00:00,16112.732845,7123.501058,4546.560557,4924.150782,2703.793985,10919.081096,7241.729856,4220.0,9104.457279,5156.085756,2170.905416,8956.761692,1245.372883,3439.273356,1893.956134,677.637306
2016-01-01 02:00:00,17131.541620,7689.841058,4905.408069,5119.587485,2828.945516,11599.590200,7559.615842,4593.0,9650.763719,5242.995467,2238.006684,8905.708040,1245.918269,3549.860557,1900.987874,706.229597
2016-01-01 03:00:00,16985.565619,7625.473967,5225.498875,5780.330305,3158.400588,12671.614215,8492.584178,4607.0,9545.615971,4942.154159,2202.883843,8682.388739,1215.766185,3469.044140,1861.179030,685.500186
2016-01-01 04:00:00,16497.960842,7366.668030,5187.201557,5853.746174,3185.895596,12692.505772,8590.469548,4523.0,9381.517472,5127.672966,2176.426560,8470.042612,1218.969666,3432.361530,1816.505706,664.055967


In [495]:
western_demand_v4.to_csv('western_demand_v4.csv')

### Post sanity check

In [121]:
#Total consumption of BAs
ba_all = set()
for bas in loadzone_ba.values():
    for ba in bas:
        ba_all.add(ba)
total_ba = defaultdict(int)
for ba in ba_all:
    total_ba[ba] = ba_demand[ba].sum()
print(sum(total_ba.values()))
total_v4 = defaultdict(int)
for i in range(201,217):
    total_v4[western.id2zone[i]] = western_demand_v4[i].sum()
print(sum(total_v4.values()))

728952244.51
728952244.51


In [38]:
# {201: 'Washington',
#  202: 'Oregon',
#  203: 'Northern California',
#  204: 'Bay Area',
#  205: 'Central California',
#  206: 'Southwest California',
#  207: 'Southeast California',
#  208: 'Nevada',
#  209: 'Arizona',
#  210: 'Utah',
#  211: 'New Mexico Western',
#  212: 'Colorado',
#  213: 'Wyoming',
#  214: 'Idaho',
#  215: 'Montana Western',
#  216: 'El Paso'}

189397069.77069607

In [122]:
total_v4['Northern California'] + total_v4['Bay Area'] + total_v4['Central California'] + total_v4['Southwest California'] + total_v4['Southeast California']

285711534.1884375

In [123]:
total_ba['CISO'] + total_ba['PACW']*ba_loadzone_frac['PACW_CAnorth'] + total_ba['BPAT']*ba_loadzone_frac['BPAT_CAnorth'] + total_ba['IID'] + total_ba['LDWP'] + total_ba['BANC'] + total_ba['TIDC']

285711534.1884376

In [124]:
total_ba['NEVP'] == total_v4['Nevada']

True

In [125]:
total_v4['Arizona']

84424587.9887119

In [126]:
total_ba['WALC'] + total_ba['AZPS'] + total_ba['GRIF'] + total_ba['HGMA'] + total_ba['DEAA'] + total_ba['GRMA'] + total_ba['SRP'] + total_ba['TEPC'] + total_ba['PNM']*ba_loadzone_frac['PNM_AZ']

84424587.9887119

In [127]:
total_v4['Utah']

38965417.51097525

In [128]:
total_ba['PACE']*ba_loadzone_frac['PACE_UT']

38965417.51097526

In [129]:
total_v4['New Mexico Western']

16726355.62836313

In [130]:
total_ba['PNM']*ba_loadzone_frac['PNM_NM'] + total_ba['EPE']*ba_loadzone_frac['EPE_NM']

16726355.62836313

In [131]:
total_v4['Wyoming']

9117523.27300373

In [132]:
total_ba['PACE']*ba_loadzone_frac['PACE_WY'] + total_ba['WACM']*ba_loadzone_frac['WACM_WY']

9117523.27300373

In [133]:
total_v4['Idaho']

24993464.152199246

In [134]:
total_ba['PACE']*ba_loadzone_frac['PACE_ID'] + total_ba['IPCO']*ba_loadzone_frac['IPCO_ID'] + total_ba['BPAT']*ba_loadzone_frac['BPAT_ID'] + total_ba['AVA']*ba_loadzone_frac['AVA_ID']

24993464.15219924

In [135]:
total_v4['Montana Western']

13041690.646398002

In [136]:
total_ba['WWA'] + total_ba['BPAT']*ba_loadzone_frac['BPAT_MT'] + total_ba['GWA'] + total_ba['NWMT'] + total_ba['WAUW']

13041690.646398002

In [137]:
total_v4['Colorado']

62168791.256887406

In [138]:
total_ba['WACM']*ba_loadzone_frac['WACM_CO'] + total_ba['PSCO']

62168791.256887406

In [139]:
total_ba['EPE']

8397099.5

In [141]:
total_v4['El Paso']

6002307.892924974

### Post adjustment based on external references

#### Goal: 
##### Increase annual total demand in WY by 7,382,477, meanwhile, decrease annual total demand in CO by 7,368,791
##### Increase annual total demand in ID by ?, meanwhile, decrease annual total demand in UT by 8,765,418
##### Increase annual total demand in NM by 6,273,644, meanwhile, decrease annual total demand in El Paso by ? (not feasible! No change)

#### Observation:
##### PACE operates between ID, WY, UT
##### WACM operates between WY, CO
##### EPE operates between NM, El Paso 

In [204]:
ba_loadzone_frac_post_adjust = ba_loadzone_frac.copy()

In [205]:
# ba_loadzone_frac['WACM_WY'] == 0.24776448949692959
# ba_loadzone_frac['WACM_CO'] == 0.7522355105030705
delta_WY_CO = 7.382*10**6/total_ba['WACM']
# delta == 0.2913939123181348
ba_loadzone_frac_post_adjust['WACM_WY'] += delta_WY_CO
ba_loadzone_frac_post_adjust['WACM_CO'] -= delta_WY_CO

In [206]:
# ba_loadzone_frac['PACE_ID'] == 0.10190918451405982
# ba_loadzone_frac['PACE_UT'] == 0.8356703007903719
delta_ID_UT = 8.765*10**6/total_ba['PACE']
# delta == 0.1879782292686714
ba_loadzone_frac_post_adjust['PACE_ID'] += delta_ID_UT
ba_loadzone_frac_post_adjust['PACE_UT'] -= delta_ID_UT

In [207]:
# ba_loadzone_frac['EPE_NM'] == 0.28519271530306717
# ba_loadzone_frac['EPE_ElPaso'] == 0.7148072846969329
delta_NM_ElPaso = 6.274*10**6/total_ba['EPE']
# delta_NM_ElPaso == 0.7471627554252513 > ba_loadzone_frac['EPE_ElPaso'] infeasible to adjust!

In [208]:
ba_demand_file = 'BA_fixed_demand_threshold_3.csv'
ba_demand = pd.read_csv(dir_files + '/' + ba_demand_file, header = 0, parse_dates = True, index_col = 'UTC Time')
ba_demand.fillna(0, inplace=True)

western_demand_v4_adjust = pd.DataFrame(index = ba_demand.index.copy())
for i in range(201,217):
    western_demand_v4_adjust[i] = 0
col = 200
for lz,bas in loadzone_ba_rename_by_frac.items():
    col += 1
    for ba in bas:
        if ba in ba_loadzone_frac_post_adjust:
            ba_name = ba.split('_')[0]
            fraction = ba_loadzone_frac_post_adjust[ba]
            western_demand_v4_adjust[col] += ba_demand[ba_name]*fraction
        else:
            western_demand_v4_adjust[col] += ba_demand[ba]
western_demand_v4_adjust.to_csv('western_demand_v4_adjust.csv')

In [209]:
#Total consumption of BAs
ba_all = set()
for bas in loadzone_ba.values():
    for ba in bas:
        ba_all.add(ba)
total_ba = defaultdict(int)
for ba in ba_all:
    total_ba[ba] = ba_demand[ba].sum()
print(sum(total_ba.values()))
total_v4 = defaultdict(int)
for i in range(201,217):
    total_v4[western.id2zone[i]] = western_demand_v4_adjust[i].sum()
print(sum(total_v4.values()))

728952244.51
728952244.51


In [210]:
total_v4['Northern California'] + total_v4['Bay Area'] + total_v4['Central California'] + total_v4['Southwest California'] + total_v4['Southeast California']

285711534.1884375

In [211]:
total_ba['WACM']*ba_loadzone_frac_post_adjust['WACM_CO'] + total_ba['PSCO']

54786791.256887406

In [212]:
total_ba['PACE']*ba_loadzone_frac_post_adjust['PACE_WY'] + total_ba['WACM']*ba_loadzone_frac_post_adjust['WACM_WY']

16499523.27300373

In [213]:
total_v4['Washington'] + total_v4['Oregon'] + total_v4['Idaho'] + total_v4['Montana Western']

198162069.77069607